In [784]:
import os
import re
import pandas as pd

In [785]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [786]:
df = pd.read_parquet(os.path.join("data","cnb_ceska_proza.parquet"))
len(df)

47419

In [ ]:
df = df.merge(pd.read_parquet(os.path.join("data","cnb_vyber.parquet")), left_index=True, right_index=True, how='inner')

In [ ]:
df = df[df['rok'] >= 2020]

In [ ]:
len(df)

In [ ]:
df = df.explode('020_a')

In [ ]:
def isbn(x):
    try:
        if len(x.replace('-','')) == 13:
            return str(x.replace('-',''))
    except:
        pass

In [ ]:
df['isbn'] = df['020_a'].apply(lambda x: isbn(x))

In [ ]:
df[['020_a','isbn']].sample(20)

In [ ]:
odkud = "data_raw/ebooky"

In [ ]:
stazene = set([x.split(".")[0] for x in os.listdir(odkud)])
len(stazene)

In [ ]:
df = df[df['isbn'].isin(stazene)]

In [ ]:
df

In [ ]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [ ]:
autority = autority.merge(pd.read_parquet(os.path.join('data','wikidata.parquet')), left_index=True, right_index=True, how="outer")

In [ ]:
autority.sample(5)

In [ ]:
df = df.merge(autority[['w_gender']], left_on="autorstvo_kod", right_index=True, how="left")

In [ ]:
df.sample(5)

In [ ]:
len(df)

In [ ]:
len(df[df['w_gender'].notnull()])

In [ ]:
import simplemma

In [ ]:
def remove_non_letters(input_string):
        # Define an empty string to store our result
        result_string = ''

        # Loop through each character in the given string
        for character in input_string:
            # If the character is a letter or whitespace
            if character.isalpha() or character.isspace():
                # Include it in our result string
                result_string += character

        # Return the cleaned string
        return result_string

In [ ]:
ukazky = []
for i in df[df['w_gender'].notnull()]['isbn'].to_list():
    ukazka = {}
    ukazka['isbn'] = i
    with open(os.path.join(odkud, f"{i}.txt"), 'r', encoding='utf-8') as u:
        text = remove_non_letters(u.read().lower().strip().replace('\n',' ').replace('\t','').replace('\xa0',' '))
        slova = []
        for t in [t for t in text.split(' ') if len(t) > 2]:
            slova.append(simplemma.lemmatize(t, lang='cs'))
        ukazka['slova'] = set(slova)
        if len(set(slova)) > 50:
            ukazky.append(ukazka)

In [ ]:
mz = pd.DataFrame(ukazky)
mz.sample(5)

In [ ]:
mz = mz.merge(df[['isbn','w_gender','autorstvo_kod']], left_on='isbn', right_on='isbn', how='left')

In [ ]:
mz

In [ ]:
len(mz[mz['w_gender'] == 'muž'])

In [ ]:
mz[mz['w_gender'] == 'muž']['autorstvo_kod'].nunique()

In [ ]:
len(mz[mz['w_gender'] == 'žena'])

In [ ]:
mz[mz['w_gender'] == 'žena']['autorstvo_kod'].nunique()

In [ ]:
mz_slova = mz.explode('slova')
mz1000 = mz_slova.groupby('slova').size().nlargest(2953).index.to_list()
mz_slova[mz_slova['slova'] == mz1000[-1]]['isbn'].nunique() / mz_slova['isbn'].nunique()

In [ ]:
mz_filtr = mz.explode('slova')
mz_filtr = mz_filtr[mz_filtr['slova'].isin(mz1000)]
mz_filtr.sample(10)

In [ ]:
z = mz_filtr[mz_filtr['w_gender'] == 'žena'].groupby('slova').size() / mz_filtr[mz_filtr['w_gender'] == 'žena']['isbn'].nunique()

In [ ]:
z.sort_values()

In [ ]:
m = mz_filtr[mz_filtr['w_gender'] == 'muž'].groupby('slova').size() / mz_filtr[mz_filtr['w_gender'] == 'muž']['isbn'].nunique()

In [ ]:
m.sort_values()

In [ ]:
pomer = m / z

## Celkový pohled

In [ ]:
pomer.plot.hist(bins=20)

In [ ]:
neutralita = pd.DataFrame(pomer).rename(columns={0:'cetnost'})
neutralita['odchylka'] = neutralita['cetnost'].apply(lambda x: x-1 if x>1 else 1-x)
neutralita.sort_values(by='odchylka').head(20)

## Čím knihy častěji začínají muži-spisovatelé

In [ ]:
pomer.nlargest(25)

## Čím knihy častěji začínají ženy-spisovatelky

In [ ]:
(1/pomer).nlargest(25)